In [14]:
import pandas as pd
import numpy as np

In [15]:
data = pd.read_csv("data/Uefa Euro Cup All Matches.csv")

In [16]:
data.head(1)

,Date,Time,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,Stage,SpecialWinConditions,Stadium,City,Attendance,Year
0,6 July 1960 (1960-07-06),20:00,France,Yugoslavia,4,5,Semi-finals,NaN,Parc des Princes,Paris,26370,1960


Date (with Time), Home Team, Away Team, HGoals, AGoals

In [17]:
# https://regexr.com/
# https://strftime.org/
# https://www.kaggle.com/c/ncaam-march-mania-2021

In [18]:
captured_date = (data['Date'].str.extract(r"\((.*)\)").squeeze() + " " 
                 + data['Time'].str.extract(r"(\d+\:\d+)").squeeze())
captured_date

0      1960-07-06 20:00
1      1960-07-06 21:30
2      1960-07-09 21:30
3      1960-07-10 21:30
4      1964-06-17 20:00
             ...       
281    2016-07-02 21:00
282    2016-07-03 21:00
283    2016-07-06 21:00
284    2016-07-07 21:00
285    2016-07-10 21:00
Name: 0, Length: 286, dtype: object

In [19]:
captured_date = pd.to_datetime(captured_date, format="%Y-%m-%d %H:%M")
clean_data = pd.DataFrame({"date": captured_date})
clean_data.head()

,date
0,1960-07-06 20:00:00
1,1960-07-06 21:30:00
2,1960-07-09 21:30:00
3,1960-07-10 21:30:00
4,1964-06-17 20:00:00


In [20]:
match_data = data[['HomeTeamName', 'AwayTeamName', 'HomeTeamGoals', 'AwayTeamGoals']]
clean_data_merged = pd.concat([clean_data, match_data], axis=1)

In [21]:
clean_data_merged

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals
0,1960-07-06 20:00:00,France,Yugoslavia,4,5
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1
4,1964-06-17 20:00:00,Spain,Hungary,2,1
...,...,...,...,...,...
281,2016-07-02 21:00:00,Germany,Italy,1,1
282,2016-07-03 21:00:00,France,Iceland,5,2
283,2016-07-06 21:00:00,Portugal,Wales,2,0
284,2016-07-07 21:00:00,Germany,France,0,2


In [22]:
def decide_y(home_goals, away_goals):
    if home_goals > away_goals:
        return 1
    elif home_goals < away_goals:
        return -1
    else:
        return 0
    
labels = np.zeros(clean_data_merged.shape[0])
for i, (home_goals, away_goals) in enumerate(clean_data_merged[['HomeTeamGoals', 'AwayTeamGoals']].values): 
    if home_goals > away_goals:
        labels[i] = 1
    elif home_goals < away_goals:
        labels[i] = -1
    else:
        labels[i] = 0
clean_data_merged['y_classif'] = labels

In [23]:
clean_data_merged['y_reg'] = clean_data_merged['HomeTeamGoals'] - clean_data_merged['AwayTeamGoals']

In [24]:
clean_data_merged

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg
0,1960-07-06 20:00:00,France,Yugoslavia,4,5,-1.0,-1
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3,-1.0,-3
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0,1.0,2
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1,1.0,1
4,1964-06-17 20:00:00,Spain,Hungary,2,1,1.0,1
...,...,...,...,...,...,...,...
281,2016-07-02 21:00:00,Germany,Italy,1,1,0.0,0
282,2016-07-03 21:00:00,France,Iceland,5,2,1.0,3
283,2016-07-06 21:00:00,Portugal,Wales,2,0,1.0,2
284,2016-07-07 21:00:00,Germany,France,0,2,-1.0,-2


p > 1 - time da casa ganhou  
-1 < p < 1 - empate  
p < -1 - time de fora ganhou

In [26]:
clean_data_merged.to_parquet("./data/clean_data_merged_v1.parquet")